## 2. ขั้นตอนการสกัดข้อความ

### ขั้นตอนที่ 1: แยกเสียงออกจากวิดีโอ
ใช้ moviepy เพื่อดึงเสียง:

In [ ]:
from moviepy import VideoFileClip

# โหลดไฟล์วิดีโอ
# video = VideoFileClip("video.mp4")
print("โหลดไฟล์วิดีโอ")
video = VideoFileClip("../sources/vdo/20241129_095733.mp4")
# video = VideoFileClip("../sources/vdo/20241129_111233.mp4")

# ดึงเสียงออกจากวิดีโอ
print("ดึงเสียงออกจากวิดีโอ")
audio = video.audio

# บันทึกไฟล์เสียง
# audio.write_audiofile("audio.wav")
print("บันทึกไฟล์เสียง")
audio.write_audiofile("../sources/audio/audio.wav")

### ขั้นตอนที่ 2: ลดเสียงรบกวน
ใช้ noisereduce เพื่อลด noise:

In [ ]:
import noisereduce as nr
import numpy as np
import scipy.io.wavfile as wav

# โหลดไฟล์เสียง
rate, data = wav.read("../sources/audio/audio.wav")

# เลือกตัวอย่าง noise (ช่วงที่ไม่มีคำพูด)
noise_sample = data[:5000]

# ลดเสียงรบกวน
reduced_noise = nr.reduce_noise(y=data, sr=rate, y_noise=noise_sample)

# บันทึกเสียงใหม่
wav.write("../sources/audio/clean_audio.wav", rate, reduced_noise.astype(np.int16))


### ขั้นตอนที่ 3: แปลงเสียงเป็นข้อความ
ใช้ speech_recognition:

In [ ]:
import speech_recognition as sr

# โหลดไฟล์เสียงที่ผ่านการลบ noise แล้ว
recognizer = sr.Recognizer()
with sr.AudioFile("clean_audio.wav") as source:
    audio = recognizer.record(source)

# ใช้ Google Speech Recognition
try:
    text = recognizer.recognize_google(audio, language="th-TH")
    print("ข้อความที่ได้:", text)
except sr.UnknownValueError:
    print("ไม่สามารถเข้าใจเสียงได้")
except sr.RequestError:
    print("ไม่สามารถเชื่อมต่อกับบริการได้")


## 3. การปรับปรุงคุณภาพเสียง (Optional)
แปลงเสียงเป็น mono: ใช้ pydub เพื่อลดความซับซ้อนของเสียง:
ปรับอัตราการสุ่มตัวอย่าง (Resampling): ลด sampling rate เพื่อปรับคุณภาพเสียงให้เหมาะกับโมเดล.

In [3]:
from pydub import AudioSegment

# โหลดเสียง
audio = AudioSegment.from_file("../sources/audio/audio.wav")

# แปลงเป็น mono
audio = audio.set_channels(1)
audio.export("../sources/audio/mono_audio.wav", format="wav")


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


<_io.BufferedRandom name='../sources/audio/mono_audio.wav'>

## 4. ข้อแนะนำ
Noise Removal: หากเสียงรบกวนมาก ควรใช้เครื่องมือเฉพาะ เช่น Audacity เพื่อลด noise ก่อน
Custom Models: สำหรับภาษาไทย อาจพิจารณาใช้ VOSK หรือบริการคลาวด์ เช่น Google Speech-to-Text API ที่รองรับภาษาไทยได้ดี
Cloud Solutions: ใช้บริการคลาวด์สำหรับการประมวลผลเสียงขนาดใหญ่ เช่น AWS Transcribe, Google Cloud, หรือ Microsoft Azure